In [11]:
import nltk
import gensim
from gensim.models import Word2Vec

In [12]:
nltk.download('gutenberg')
gutenberg = nltk.corpus.gutenberg.sents()

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\herbe\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [13]:
model = Word2Vec(gutenberg, vector_size=100, window=5, workers=4)
model.save("word2vec.model")

In [14]:
def predict_next_word(input_list,model):
    vector_odds = model.wv.most_similar(positive=input_list, topn=5)
    return vector_odds

In [15]:
loaded_model = Word2Vec.load("word2vec.model")

In [16]:
input = ["The", "book", "is"]
possibilities = predict_next_word(input,loaded_model)

In [17]:
print("Next word: ", possibilities[0][0])
print("Possibilities: ", possibilities)

Next word:  whaling
Possibilities:  [('whaling', 0.5997612476348877), ('testament', 0.564294159412384), ('sight', 0.5448867678642273), ('Sperm', 0.5392782092094421), ('chronicles', 0.5373831391334534)]
